In [13]:
import tensorflow as tf
import keras
from keras.layers import Layer, Input, Dense, Conv3D, MaxPooling3D, UpSampling3D, Dropout, Flatten,InputLayer , Reshape, concatenate, Concatenate
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
import numpy as np

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

loss_history = LossHistory()
        
input_voxel = Input(shape=(32, 32, 16, 1))
scales = Input(shape=(1, 3))

x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(input_voxel)
x = MaxPooling3D((2, 2, 2), padding='same')(x)
x = Conv3D(64, (3, 3 ,3), activation='relu', padding='same')(x)
x = MaxPooling3D((2, 2, 2), padding='same')(x)
x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)

x = Flatten()(x)
x = Reshape((1, 16384))(x)
x = Concatenate()([x, scales])


x = Dense(512)(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
encoded = Dense(64)(x)


x = Dense(8192)(encoded)
x = Reshape((8, 8, 4, 32))(x)
x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(x)
x = UpSampling3D((2, 2, 2))(x)
x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(x)
x = UpSampling3D((2, 2, 2))(x)
decoded = Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same')(x)


e_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1)


def custom_loss(y_true, y_pred):
    loss_c = -tf.math.reduce_mean(keras.losses.binary_crossentropy(y_true, y_pred))
    
    FN_TO_FP_WEIGHT = 0.9
    loss_r = - tf.math.reduce_mean(FN_TO_FP_WEIGHT * y_true * keras.backend.log(y_pred + 1e-10) + (1 - FN_TO_FP_WEIGHT) * \
                                (1 - y_true) * keras.backend.log(1 - y_pred + 1e-10))
    
    LOSS_R_WEIGHT = 200
    LOSS_C_WEIGHT = 1
    loss = LOSS_C_WEIGHT * loss_c + LOSS_R_WEIGHT * loss_r
    
    return loss

autoencoder = Model([input_voxel, scales], decoded)
autoencoder.compile(optimizer='adadelta', loss=custom_loss)

history = autoencoder.fit(x=[np.zeros((1, 32, 32, 16, 1)), np.zeros((1, 1, 3))], y=np.zeros((1, 32, 32, 16, 1)), epochs=50, batch_size=1)
# autoencoder.save("./keras_models/model.h5")
# with open('./keras_models/losses.txt', 'w') as f:
#     for item in loss_history.losses:
#         f.write("%s\n" % item)
        
        

Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 13.1696
Epoch 2/50
1/1 [==============================] - 0s 179ms/step - loss: 13.1564
Epoch 3/50
1/1 [==============================] - 0s 174ms/step - loss: 13.1428
Epoch 4/50
1/1 [==============================] - 0s 180ms/step - loss: 13.1290
Epoch 5/50
1/1 [==============================] - 0s 175ms/step - loss: 13.1153
Epoch 6/50
1/1 [==============================] - 0s 178ms/step - loss: 13.1014
Epoch 7/50
1/1 [==============================] - 0s 173ms/step - loss: 13.0877
Epoch 8/50
1/1 [==============================] - 0s 187ms/step - loss: 13.0738
Epoch 9/50
1/1 [==============================] - 0s 201ms/step - loss: 13.0599
Epoch 10/50
1/1 [==============================] - 0s 177ms/step - loss: 13.0461
Epoch 11/50
1/1 [==============================] - 0s 173ms/step - loss: 13.0319
Epoch 12/50
1/1 [==============================] - 0s 183ms/step - loss: 13.0179
Epoch 13/50
1/1 [=======================